Importing the libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
tf.__version__

'2.12.0'

Part-I Data Preprocessing

Importing datasets

In [ ]:
dataset = pd.read_csv('Churn_Modelling.csv')
X=dataset.iloc[:,3:-1].values
Y=dataset.iloc[:,-1].values

In [ ]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [ ]:
print(Y)

[1 0 1 ... 1 1 0]


## **ENCODING CATEGORICAL DATA**

Label Encoder the Gender column

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])

In [ ]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot encoding the Geography column

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])],remainder='passthrough')
X=np.array(ct.fit_transform(X))

In [ ]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


## **Splitting the dataset into training set and test set**

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=0)

### **Feature Scaling**

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train=sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# **BUILDING** the ANN  ->


Initializing the ANN

In [ ]:
ann = tf.keras.models.Sequential()

Adding the input layer and the first hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

Adding second layer


In [ ]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

Adding output layer


In [ ]:
ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

**PART 3 :TRAINING the ANN**

COMPILING THE ANN

In [ ]:
ann.compile(optimizer= 'adam' ,loss= 'binary_crossentropy',metrics=['accuracy'])

TRAINING The Ann using the train set

In [ ]:
ann.fit(X_train,Y_train,batch_size = 32,epochs=100)

Epoch 1/100
250/250 [==============================] - 3s 5ms/step - loss: 0.6838 - accuracy: 0.5799
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4961 - accuracy: 0.7956
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4587 - accuracy: 0.7958
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4358 - accuracy: 0.7993
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4185 - accuracy: 0.8136
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4028 - accuracy: 0.8284
Epoch 7/100
250/250 [==============================] - 0s 988us/step - loss: 0.3870 - accuracy: 0.8382
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3741 - accuracy: 0.8443
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3657 - accuracy: 0.8491
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3600 - accu

**Part 4 : MAke predictions and evaluating the model**

Predicting the result of a single observation

In [ ]:
print(ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]]))>0.5)

1/1 [==============================] - 0s 84ms/step
[[False]]


Predicting the Test set *results*

In [ ]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1),Y_test.reshape(len(Y_test),1)),1))

63/63 [==============================] - 0s 781us/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


Making the Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm=confusion_matrix(Y_test,y_pred)
print(cm)
accuracy_score(Y_test,y_pred)

[[1508   87]
 [ 202  203]]


0.8555